In [21]:
import json
import numpy as np
import os
import pandas as pd
import re
import requests

# Some global constants
url_base = "https://futurice.com/"
file_base = str(os.path.dirname(os.path.dirname(os.path.abspath("regex_scrapper_legacy")))) + "/data/"
sep = ';;;'  # separator for csv files

In [22]:
# download the raw text from url and return it as utf-8 string
def download_raw(url):
    headers = {"User-Agent" : "Mozilla/5.0 (Windows NT 5.1; rv:7.0.1) Gecko/20100101 Firefox/7.0.1"} 
    r_temp  = requests.get(url_base + url)#, headers)

    string = bytearray()

    for chunk in r_temp.iter_content(chunk_size=128):
        string.extend(chunk)

    return string.decode(encoding='utf-8')
            

In [23]:
# scrapes the text from input string and parses most of the unnecessary characters and marking out
def process_blog_post(string):
    
    texts = ""

    for item in re.findall('"text":".*?"}', string):
        
        text = item[len('"text":"'):-1]
        #print(text)

        # Idunno why this works but removing only the \n ones doesn't
        text = re.sub('\\\\n',  " ", text)
        text = re.sub('\\\n',  " ", text)
        text = re.sub('\\n',  "", text)
        text = re.sub('\n',  "", text)
        text = re.sub("\'s", "'s", text)
        text = re.sub("\'", "", text)
        text = re.sub("#", "", text)

        # Removing unnecessary unicodes?
        #text = re.sub("\\u.*? ", " ", text)

        text = re.sub("\((http.*?|s://)\)", "", text)
        text = re.sub("(s://.*?)", "", text)
        text = re.sub("!\[.*?\]\(.*?\)", "", text)


        text = re.sub("\[", "", text)
        text = re.sub("\]", "", text)
        
        texts += text

    return texts

In [24]:
def get_blog_links(string):
    
    ret = []
    for i in re.findall(r'blog/[a-zA-Z0-9/-]+[a-zA-Z0-9]+', string):
        if i not in ret:
            ret.append(i)
        
    return ret

In [25]:
def get_title(string): 
    #print("title: " + str(re.search('"title":".*?"', string).group()[len('"title":"'):-1]))
    temp = re.search('</title>.*?content=".*?"', string)
    if temp == None:
        print("----Error: title not found")
        return ""
    else:
        return temp.group()[len(re.search('</title>.*?content="', string).group()):-1]
    
def get_teaser(string):
    #print("teaser: " + str(re.search('"teaser":".*?"', string).group()[len('"teaser":"'):-1]))
    temp = re.search('"teaser":".*?"', string)
    if temp == None:
        print("----Error: teaser not found")
        return ""
    else:
        return temp.group()[len('"teaser":"'):-1]

def get_datetime(string):
    temp = re.search('<time dateTime=".*?"', string)
    if temp == None:
        print("----Error: date not found")
        return ""
    else:
        return temp.group()[len('<time dateTime="'):-1]

def get_category(string):
    temp = re.search('<span class="sc-7f8efa2d-1 gwaujb">.*?</span>', string)
    if temp == None:
        print("----Error: category not found")
        return ""
    else:
        return temp.group()[len('<span class="sc-7f8efa2d-1 gwaujb">'):-len('</span>')]

In [26]:
# takes string and exports it to file. If csv = True, appends it as csv row to file_name, otherwise creates
# a new json file. Returns a dictonary with found values.
def process_file_to_data(string, url, csv = False, file_name = "blog_text.csv"):
    
    title = get_title(string)
    teaser = get_teaser(string)
    time  = get_datetime(string)
    category = get_category(string)
    text = process_blog_post(string)
    
        
    if not csv:
        with open(file_base + url, "w", encoding='utf-8') as fd:
            json.dump({
                "url" : url,
                "title" :title,
                "time" : time,
                "category" : category,
                "text" : text
            }, fd, indent = 4)
    else:
        with open(file_base + file_name, "a", encoding='utf-8') as fd:
            fd.write(url + sep + title + sep + time + sep + category + sep + text + "\n")
    
    return {    "url" : url,
                "title" :title,
                "time" : time,
                "category" : category,
                "text" : text}

In [27]:
def get_base_urls():
    df = pd.read_csv(file_base + "blog_urls_all.csv", encoding = 'utf-8')
    df = df.iloc[:,0].str.replace('^/', '').tolist()
    return df
        
get_base_urls()

C:\Users\talvi\AppData\Local\Temp\ipykernel_22120\454362729.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.iloc[:,0].str.replace('^/', '').tolist()


['blog/the-importance-of-customer-focus',
 'blog/futustories-six-reasons-pasi-left-and-came-back-to-futurice',
 'blog/foresight-methods-and-strategic-planning',
 'blog/uncertainty-in-business-volatile-market',
 'blog/futustories-emma-leena-heikkinens-story',
 'blog/safe-route-uncertain-times',
 'blog/data-provides-the-key-to-sustainability',
 'blog/mob-programming',
 'blog/futustories-meet-luisa-senior-software-engineer-and-tech-advisor',
 'blog/futustories-a-day-at-futurice-with-seth-peters',
 'blog/microsoft-azure-immersion-workshops-h2-2022',
 'blog/futurice-sunray-finnish-midsummer',
 'blog/corporate-foresight',
 'blog/five-things-we-have-done-to-support-mental-health-in-the-workplace',
 'blog/futurice-academy-opens-the-door-to-lifelong-learning',
 'blog/accessible-digital-services-are-an-essential-part-of-an-inclusive-world',
 'blog/future-of-mobility',
 'blog/mlops-what-is-it-and-what-can-you-do-with-it',
 'blog/futustories-meet-dariia-ux-ui-design',
 'blog/what-is-sre-and-how-do

In [28]:
def base_loading(max_num = 900, to_csv = True, create_new = True):

    unread_urls = get_base_urls()
    linked = dict()
    count = 0


    if to_csv and create_new:
        # Writing labels to file
        file_name = "blog_text.csv"
        with open(file_base + file_name, "w", encoding='utf-8') as fd:
            fd.write("url" + sep + "title" + sep + "time"+ sep  + "category"+ sep  + "text" + "\n")

    #Looping through urls until max_num is reached or there is nothing in the unread_urls
    while len(unread_urls) > 0 and count < max_num:
        count += 1
        url = unread_urls.pop(0)
        print(str(count) + ": " + url)

        string = download_raw(url)
        temp_dict = process_file_to_data(string, url, csv = to_csv)

        temp_array = []
        for item in get_blog_links(string):
            temp_array.append(item)
        linked[url] = temp_array

    if not to_csv:
        # Writing a base file with information of all the url/names of the blog text files        
        with open(file_base + "urls.csv", "w", encoding='utf-8') as fd:
            for item in read_urls:
                fd.write(item)
                fd.write("\n")

    # saves the urls of the unread ones, if one doesn't want to run all the samples in one go        
    if len(unread_urls) > 0:
        with open(file_base + "unread.csv", "w", encoding='utf-8') as fd:
            for item in unread_urls:
                fd.write(item)
                fd.write("\n")

    # Saving to json file, which blog post linked to which other      
    if len(linked) > 0:
        with open(file_base + "linked.json", "w", encoding='utf-8') as fd:
            json.dump(linked, fd, indent = 4)

    print("Finished")

In [29]:
base_loading()

C:\Users\talvi\AppData\Local\Temp\ipykernel_22120\454362729.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df = df.iloc[:,0].str.replace('^/', '').tolist()


1: blog/the-importance-of-customer-focus
2: blog/futustories-six-reasons-pasi-left-and-came-back-to-futurice
3: blog/foresight-methods-and-strategic-planning
4: blog/uncertainty-in-business-volatile-market
5: blog/futustories-emma-leena-heikkinens-story
6: blog/safe-route-uncertain-times
7: blog/data-provides-the-key-to-sustainability
8: blog/mob-programming
9: blog/futustories-meet-luisa-senior-software-engineer-and-tech-advisor
10: blog/futustories-a-day-at-futurice-with-seth-peters
11: blog/microsoft-azure-immersion-workshops-h2-2022
12: blog/futurice-sunray-finnish-midsummer
13: blog/corporate-foresight
14: blog/five-things-we-have-done-to-support-mental-health-in-the-workplace
15: blog/futurice-academy-opens-the-door-to-lifelong-learning
16: blog/accessible-digital-services-are-an-essential-part-of-an-inclusive-world
17: blog/future-of-mobility
18: blog/mlops-what-is-it-and-what-can-you-do-with-it
19: blog/futustories-meet-dariia-ux-ui-design
20: blog/what-is-sre-and-how-does-it-r

In [31]:
df = pd.read_csv(file_base + "blog_text.csv", sep = sep, engine='python', encoding = 'utf-8', names = ["url", "title", "timedate", "category", "text"])
df

,url,title,timedate,category,text
0,url,title,time,category,text
1,blog/the-importance-of-customer-focus,Why are we still talking about customer focus?...,2022-09-23T00:00:00.000Z,Opinion,What is customer focus – and why is it so eas...
2,blog/futustories-six-reasons-pasi-left-and-cam...,"For Senior Cloud Consultant Pasi, a change can...",2022-09-16T00:00:00.000Z,Culture,1. I need awesome people around me… I’d say ...
3,blog/foresight-methods-and-strategic-planning,Foresight methods and strategic planning lead ...,2022-09-13T00:00:00.000Z,Strategy,This is where foresight methods and strategic ...
4,blog/uncertainty-in-business-volatile-market,"Future uncertainty, how to deal with uncertain...",2022-09-12T00:00:00.000Z,Opinion,The silver lining to all this doom and gloom i...
...,...,...,...,...,...
814,blog/make-up-simple-solutions-that-fit-your-or...,Hello Mikko Viikari! You made a presentation a...,2010-05-10T12:20:27.000+03:00,Culture,__Hello Mikko Viikari! You made a presentation...
815,blog/pekka-tarjanne-1937-2010,NaN,2010-03-19T11:34:28.000+02:00,News,!Pekka With deep sorrow we announce the loss ...
816,blog/user-testing-the-ultimate-reality-check,"Last week, after a couple of months of design ...",2010-03-12T11:36:49.000+02:00,Ways of Working,"Last week, after a couple of months of design ..."
817,blog/quality-time-session-based-testing-faq,NaN,2010-02-26T11:38:23.000+02:00,Learning,!quality-time-logo-e1263459716150 Session ...


In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 819 entries, 0 to 818
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   url       819 non-null    object
 1   title     653 non-null    object
 2   timedate  791 non-null    object
 3   category  787 non-null    object
 4   text      810 non-null    object
dtypes: object(5)
memory usage: 32.1+ KB
